In [1]:
import os
import webbrowser
import time
import difflib
import threading
import subprocess
import uvicorn
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel
from typing import List, Dict
import requests


# Function Registry

def open_chrome():
    """Opens Google Chrome by launching the default browser to Google."""
    try:
        webbrowser.open("https://www.google.com")
        print("Chrome opened successfully.")
    except Exception as e:
        print(f"Error opening Chrome: {e}")

def open_calculator():
    """Opens Calculator (Windows-specific)."""
    try:
        os.system("calc")  # Note: On non-Windows systems, replace with an appropriate command.
        print("Calculator opened successfully.")
    except Exception as e:
        print(f"Error opening Calculator: {e}")

def open_notepad():
    """Opens Notepad (Windows-specific)."""
    try:
        os.system("notepad")
        print("Notepad opened successfully.")
    except Exception as e:
        print(f"Error opening Notepad: {e}")

# Register the functions with keywords for retrieval
FUNCTION_REGISTRY: Dict[str, Dict] = {
    "chrome": {"function": open_chrome, "name": "open_chrome"},
    "calculator": {"function": open_calculator, "name": "open_calculator"},
    "notepad": {"function": open_notepad, "name": "open_notepad"}
}

# Simulated LLM + RAG Function Retrieval

def retrieve_function(user_query: str):
    """
    Simulates function retrieval using keyword matching.
    Returns a tuple: (function_reference, function_name)
    """
    query = user_query.lower()
    best_match = None
    best_ratio = 0.0
    
    for keyword, func_data in FUNCTION_REGISTRY.items():
        ratio = difflib.SequenceMatcher(None, query, keyword).ratio()
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = func_data
            
    # If no good match is found, return None values.
    if best_ratio < 0.3:
        return None, None
    return best_match["function"], best_match["name"]

# Example test:
test_query = "Launch Google Chrome"
func, func_name = retrieve_function(test_query)
print(f"Retrieved function: {func_name}")  # Expected: open_chrome (depending on the matching)

# Dynamic Code Generation for Function Invocation

def generate_code(function_name: str):
    """
    Generates a Python code snippet that calls the specified function with error handling.
    """
    code = f"""
from __main__ import {function_name}

def main():
    try:
        {function_name}()
        print("{function_name} executed successfully.")
    except Exception as e:
        print("Error executing {function_name}:", e)

if __name__ == "__main__":
    main()
"""
    return code

# Example test:
if func_name:
    code_snippet = generate_code(func_name)
    print("Generated Code Snippet:")
    print(code_snippet)

# Session-Based Context Management

session_history: List[Dict] = []

def add_to_history(user_query: str, function_name: str, code_snippet: str):
    """
    Stores the current interaction in the session history.
    """
    session_history.append({
        "timestamp": time.time(),
        "query": user_query,
        "retrieved_function": function_name,
        "code": code_snippet
    })

# Example of updating history:
if func_name:
    add_to_history(test_query, func_name, code_snippet)
    print("Session history updated:")
    print(session_history[-1])

# FastAPI Setup

app = FastAPI()

# Define the prompt model
class TaskPrompt(BaseModel):
    command: str

# Function to open Chrome
def open_chrome():
    try:
        subprocess.run(["start", "chrome"], shell=True)  # Works on Windows
        return "Chrome opened successfully!"
    except Exception as e:
        return f"Error opening Chrome: {e}"

# Function to open Notepad
def open_notepad():
    try:
        subprocess.run(["notepad"], shell=True)  # Works on Windows
        return "Notepad opened successfully!"
    except Exception as e:
        return f"Error opening Notepad: {e}"

# Function to open Calculator
def open_calculator():
    try:
        subprocess.run(["calc"], shell=True)  # Works on Windows
        return "Calculator opened successfully!"
    except Exception as e:
        return f"Error opening Calculator: {e}"

# Task handler
@app.post("/execute")
async def execute_task(task: TaskPrompt, background_tasks: BackgroundTasks):
    if task.command == "open_chrome":
        result = open_chrome()
    elif task.command == "open_notepad":
        result = open_notepad()
    elif task.command == "open_calculator":
        result = open_calculator()
    else:
        result = "Command not recognized. Available commands: open_chrome, open_notepad, open_calculator."

    # Add a background task to simulate a long-running process
    background_tasks.add_task(simulate_long_task, task.command)
    return {"message": result}

# Simulate a long-running task
def simulate_long_task(command: str):
    time.sleep(5)  # Simulate a delay
    print(f"Background task for '{command}' completed.")

# Function to run the FastAPI app
def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8004)

# Run the server in a separate thread to avoid blocking
if __name__ == "__main__":
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    time.sleep(2)  # Give the server time to start

    # Test the /execute endpoint with a sample prompt
    test_prompt = {"command": "open_calculator"}
    response = requests.post("http://127.0.0.1:8004/execute", json=test_prompt)
    if response.status_code == 200:
        print("API Response:")
        print(response.json())
    else:
        print("Error:", response.status_code, response.text)

    # Displaying Session History
    print("\nSession History:")
    for entry in session_history:
        print(entry)


INFO:     Started server process [4328]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8004 (Press CTRL+C to quit)


Retrieved function: open_chrome
Generated Code Snippet:

from __main__ import open_chrome

def main():
    try:
        open_chrome()
        print("open_chrome executed successfully.")
    except Exception as e:
        print("Error executing open_chrome:", e)

if __name__ == "__main__":
    main()

Session history updated:
{'timestamp': 1743142756.3035383, 'query': 'Launch Google Chrome', 'retrieved_function': 'open_chrome', 'code': '\nfrom __main__ import open_chrome\n\ndef main():\n    try:\n        open_chrome()\n        print("open_chrome executed successfully.")\n    except Exception as e:\n        print("Error executing open_chrome:", e)\n\nif __name__ == "__main__":\n    main()\n'}
INFO:     127.0.0.1:65476 - "POST /execute HTTP/1.1" 200 OK
API Response:
{'message': 'Calculator opened successfully!'}

Session History:
{'timestamp': 1743142756.3035383, 'query': 'Launch Google Chrome', 'retrieved_function': 'open_chrome', 'code': '\nfrom __main__ import open_chrome\n\ndef main()

Background task for 'open_calculator' completed.
